In [19]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("nbtpj/BioNLP2021")

# Display the available splits
#print("Available Splits:", dataset.keys())

# Load the training split (you can also explore 'validation' or 'test')
train_data = dataset['train_mul']

# Convert to pandas DataFrame for easier exploration
df = pd.DataFrame(train_data)

# print(df['question'])


In [37]:
import os
from dotenv import load_dotenv
import anthropic
import re

In [15]:
# Load environment variables from a .env file
load_dotenv()


True

In [16]:
# Read the API key from the environment variable
API_KEY = os.getenv("ANTHROPIC_API_KEY")
if not API_KEY:
    raise ValueError("API key not found. Please set ANTHROPIC_API_KEY in your .env file.")

# Initialize the Anthrop client
client = anthropic.Anthropic(api_key=API_KEY)


In [29]:
# Function to query the API for specific rows
def query_claude(content: str, question: str, model: str = "claude-3-5-sonnet-20241022", max_tokens: int = 8192, temperature: float = 0):
    """
    Sends a query to Claude with the given content and question.

    Args:
        content (str): The content to analyze.
        question (str): The question to answer based on the content.
        model (str): The model to use (default is 'claude-3-5-sonnet-20241022').
        max_tokens (int): Maximum tokens for the response.
        temperature (float): Temperature setting for response creativity.

    Returns:
        str: The assistant's response content.
    """
    message = client.messages.create(
        model=model,
        max_tokens=max_tokens,
        temperature=temperature,
        messages=[
            {
                "role": "user",
                "content": f"""
You are an AI assistant tasked with answering questions based solely on provided content. Your job is to analyze the given information and respond to a specific question without using any external knowledge.

Here is the content you should use to answer the question:

<content>
{content}
</content>

Now, please answer the following question:

<question>
{question}
</question>

Instructions:
1. Read the content and question carefully.
2. Use only the information provided in the content to formulate your answer. Do not use any external knowledge or information not present in the given content.
3. If you find an answer to the question within the content, provide that answer.
4. If you cannot find an answer to the question within the content, state that you couldn't find an answer based on the given information.
5. After providing your answer (or stating that you couldn't find one), indicate whether you found an answer using the format specified below.

Before providing your final answer, wrap your analysis in <analysis> tags. Consider the following:
- Write down relevant quotes from the content that might answer the question.
- Analyze each quote for its relevance and how well it answers the question.
- Summarize the key points found in the content related to the question.
- Is there a clear answer to the question in the content?
- If there isn't a clear answer, is there any information that partially addresses the question?

Output Format:
1. Provide your answer or state that you couldn't find an answer based on the given information.
2. On a new line, use the following format to indicate whether an answer was found:
```text
found_answer = 1  # if the answer was found in the content
found_answer = 0  # if the answer was not found in the content
""" } ] ) 
    return message.content

In [44]:
# Display the first few rows
counter = 0
answers = []
for row in  train_data:
    print(len(row['text']))
    print(row['question'])
    response  = query_claude(row['text'], row['question'])
    print(response[0].text)
    
    match = re.search(r"found_answer\s*=\s*(\d)", response[0].text)
    if match:
        found_answer = int(match.group(1))  # Extract and convert to integer
        answers.append(found_answer)
    else:
        print("No 'found_answer' found in the text.")

    if counter >= 10:
        break
    else:
        counter += 1

31809
abetalipoproteimemia hi, I would like to know if there is any support for those suffering with abetalipoproteinemia?<SS>I am not diagnosed but have had many test that indicate I am suffering with this, keen to learn how to get it diagnosed and how to manage, many thanks
<analysis>
Relevant quotes from the content:
1. "The Genetic Testing Registry (GTR) provides information about the genetic tests available for abetalipoproteinemia. The intended audience for the GTR is health care providers and researchers. Patients and consumers with specific questions about a genetic test should contact a health care provider or a genetics professional."

2. "Management in adults typically focuses on specific complications associated with the disorder, and depends on the signs and symptoms present. Affected people may need consultations with several other types of specialists, including a lipidologist, gastroenterologist, hepatologist, ophthalmologist, and neurologist."

3. Regarding diagnosis, 

In [45]:
answers

[1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1]

In [53]:
quality_answers = 100 * sum(answers)/len(answers)
print(f"{quality_answers:.0f}%")

82%
